In [ ]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql import functions as F
from pyspark.sql import types as T
from datetime import datetime
import sys
from pyspark.sql import Row
config = configparser.ConfigParser()
config.read("dl.cfg")

os.environ["AWS_ACCESS_KEY_ID"] = config["AWS"]["KEY"]
os.environ["AWS_SECRET_ACCESS_KEY"] = config["AWS"]["SECRET"]
#os.environ["PYSPARK_PYTHON"] = 'python3'

In [ ]:
def create_spark_session():
    spark = SparkSession.builder.config(
        "spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0"
    ).appName("sparkstand").getOrCreate()
    return spark

In [ ]:
spark = SparkSession.builder.master("spark://gambit:7077").appName("spark3").config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.1").getOrCreate()

In [ ]:
input_data = config["S3"]["IN_PATH"]
output_data = config["S3"]["OUT_PATH"]
#input_data = 'processed'
#output_data = 'processed'
start = '2013-01-04'
end = '2016-12-31'

In [ ]:
working_day = datetime.strptime(start, '%Y-%m-%d')

In [ ]:
temperature_data = "{}/temperature/{}/{}/{}/*.json".format(input_data, working_day.year, working_day.month, working_day.day)
df = spark.read.json(temperature_data)

In [ ]:
# Check 1, make sure we have 24 hours
temperature_count = df.count()

In [ ]:
avgtemp = df.agg({"Los Angeles": "avg"}).collect()[0][0]

In [ ]:
avgtemp

In [ ]:
crime_data = "{}/crimes/{}/{}/{}/*.csv".format(input_data, working_day.year, str(working_day.month).zfill(2),
                                               str(working_day.day).zfill(2))
df2 = spark.read.csv(crime_data, header=True)

In [ ]:
df2.printSchema()

In [ ]:
# data check number 2, make sure no extra dates snuck in
unique_count = df2.select('Date Occurred').distinct().count()

In [ ]:
crime_count = df2.count()

In [ ]:
crime_count

In [ ]:
avgtemp

In [ ]:
start

In [ ]:
Output = Row("date", "crimes", "temperature", "year", "month", "day")
output1 = Output(start, crime_count, avgtemp, working_day.year, working_day.month, working_day.day)
dframe = spark.createDataFrame([output1])

In [ ]:
dframe = dframe.withColumn(
        "date", F.to_date(dframe.date.cast(dataType=T.TimestampType()))
)

In [ ]:
dframe.printSchema()

In [ ]:
dframe.write.partitionBy("year", "month", "day").parquet("{}/test".format(output_data), mode="append")